In [ ]:
import open3d.ml.torch as ml3d
import argparse
import math
import numpy as np
import os
import random
import sys
import torch
from os.path import exists, join, isfile, dirname, abspath, split

In [ ]:
def pred_custom_data(pc_names, pcs, pipeline_r, pipeline_k):
    vis_points = []
    
    for i, data in enumerate(pcs):
        name = pc_names[i]

        results_r = pipeline_r.run_inference(data)
        pred_label_r = (results_r['predict_labels'] + 1).astype(np.int32)
        # WARNING, THIS IS A HACK
        # Fill "unlabeled" value because predictions have no 0 values.
        pred_label_r[0] = 0

        label = data['label']
        pts = data['point']

        vis_d = {
            "name": name,
            "points": pts,
            "labels": label,
            "pred": pred_label_r,
        }
        vis_points.append(vis_d)

        vis_d = {
            "name": name + "_randlanet",
            "points": pts,
            "labels": pred_label_r,
        }
        vis_points.append(vis_d)

    return vis_points

In [ ]:
basedir = "/home/threedee/repos/Open3D-ML"
datesetbase = "/home/threedee/datasets"

randlanet_smartlab_cfg = basedir + "/ml3d/configs/randlanet_smartlab.yml"
ckpt_path = basedir + "/mytests/logs/RandLANet_SmartLab_tf/checkpoint/ckpt-5"

kitti_labels = ml3d.datasets.SemanticKITTI.get_label_to_names()

ckpt_path = 
model = ml3d.models.SmartLab(ckpt_path=ckpt_path, )
pipeline_r = ml3d.pipelines.SemanticSegmentation(model)
pipeline_r.load_ckpt(model.cfg.ckpt_path)

pcs_with_pred = pred_custom_data(pc_names, pcs, pipeline_r, pipeline_k)

    v = ml3d.vis.Visualizer()
lut = ml3d.vis.LabelLUT()

for val in sorted(kitti_labels.keys()):
    lut.add_label(kitti_labels[val], val)

v.set_lut("labels", lut)
v.set_lut("pred", lut)

v.visualize(pcs_with_pred)